This notebook shows that Mosaic works with a serverless DuckDB using [MotherDuck](https://motherduck.com). To use this notebook, you need to have access to MotherDuck and the Gaia dataset in a specific database. We will make this notebook more general in the future.

In [ ]:
import duckdb
import yaml
from mosaic_widget import MosaicWidget
import time

In [ ]:
con = duckdb.connect("md:gaia-import")

con.execute("ATTACH ':memory:' AS local")
# con.execute("ATTACH 'indexes.db' AS local") # use precomputed indexes in case network is unstable

# switch to local db as the default for indexes
con.execute("USE local")

In [ ]:
con.query('SELECT count(*) FROM "gaia-import".gaia_sample_5_percent_projected')

In [ ]:
with open("../../dev/yaml/gaia.yaml") as f:
    spec = yaml.safe_load(f)

# use existing table
del spec["data"]

table = "gaia-import.gaia_sample_5_percent_projected"

spec["hconcat"][0]["vconcat"][0]["plot"][0]["data"]["from"] = table
spec["hconcat"][0]["vconcat"][1]["hconcat"][0]["plot"][0]["data"]["from"] = table
spec["hconcat"][0]["vconcat"][1]["hconcat"][1]["plot"][0]["data"]["from"] = table
spec["hconcat"][2]["plot"][0]["data"]["from"] = table
spec;

In [ ]:
MosaicWidget(con=con, spec=spec, temp_indexes=True)

In [ ]:
# Show the indexes
con.query("SHOW TABLES")

## Set up queries

In [ ]:
# Potentially faster writes

con.execute("set preserve_insertion_order=false")

In [ ]:
# Install MotherDuck

c = duckdb.connect()
c.query("FORCE INSTALL motherduck")

In [ ]:
# Create a table with the u and v columns with natural earth projection

c = duckdb.connect("md:gaia-import")
c.execute("""
CREATE TABLE IF NOT EXISTS gaia_sample_5_percent_projected AS
WITH tmp AS (
  SELECT
    radians((-l + 540) % 360 - 180) AS lambda,
    radians(b) AS phi,
    asin(sqrt(3)/2 * sin(phi)) AS t,
    t^2 AS t2,
    t2^3 AS t6,
    *
  FROM gaia_sample_5_percent
)
SELECT
  (1.340264 * lambda * cos(t)) / (sqrt(3)/2 * (1.340264 + (-0.081106 * 3 * t2) + (t6 * (0.000893 * 7 + 0.003796 * 9 * t2)))) AS u,
  t * (1.340264 + (-0.081106 * t2) + (t6 * (0.000893 + 0.003796 * t2))) AS v,
  * EXCLUDE('t', 't2', 't6')
FROM tmp
""")

In [ ]:
# Show all remote tables

c = duckdb.connect("md:gaia-import")
c.query("SHOW TABLES")